Caution: so far the notebook is read-only. You need to download __[flights.csv](https://www.kaggle.com/usdot/flight-delays/data)__ before running the notebook. Also run one of XGBoost, LightGBM, or CatBoost while commenting out the other two.
 
 Notebook source: Toward Data Science

In [5]:
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import threading
%matplotlib inline

train_df = pd.read_csv('train_small.csv', header=None)
test_df = pd.read_csv('test_small.csv')

In [28]:
n_train = 60000
train_targets = train_df.iloc[:n_train,1]
train_data = train_df.iloc[:n_train,2:]
test_targets = train_df.iloc[n_train:,1]
test_data = train_df.iloc[n_train:,2:]

In [25]:
import lightgbm as lgb
from sklearn import metrics

def auc2(m, train, test): 
    return (metrics.roc_auc_score(train_targets,m.predict(train)),
                            metrics.roc_auc_score(test_targets,m.predict(test)))

lg = lgb.LGBMRegressor(silent=False)
param_dist = {"max_depth": [1,2, 3],
              "learning_rate" : [0.1,0.5,1],
              # "num_leaves": [10,15,20],
              "num_leaves": [2,3],
              "n_estimators": [300]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(train_data,train_targets)
grid_search.best_estimator_

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  50 out of  54 | elapsed:   28.4s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   30.0s finished


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.5, max_depth=1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=2, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### Best predictors
500 rows 150 estimators (12s) : depth 3; rate 0.5; 10 leaves;

1000 rows 200 estimators (19s) : depth 2; rate 0.1; 10 leaves; 0.7361517358288425 auc (1 with training set)

10000 rows 300 estimators (1min1s) : depth 1; rate 0.5; 10 leaves; 0.8508816764850232 auc (0.941760012507842 with training set)

Does number of leaves impact ? NO default 31

60000 rows; 300 estiators; depth 1; rate 0.5; 10 leaves : (0.8986034603281355, 0.8795559277631042) (train then test)
60000 rows; 1000 estiators; depth 1; rate 0.5; 10 leaves : (0.9183773853908943, 0.8924150185745882) (train then test)
#### Shows that overfitting is not good ? Or just harder to have high auc with bigger amount of data.
60000 rows; 10000 estiators; depth 1; rate 0.5; 10 leaves : (0.9433394334525005, 0.8876156885481759) (train then test)
#### Now it seems clear

In [32]:
start = time.time()
lggood = lgb.LGBMRegressor(silent=False, max_depth=1, learning_rate=0.5, n_estimators=10000)
lggood.fit(train_data,train_targets)
end = time.time()
print(end-start)
auc2(lggood, train_data, test_data)

56.65645384788513


(0.9433394334525005, 0.8876156885481759)

In [30]:
lg

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.5, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [26]:
auc2(grid_search.best_estimator_, train_data, test_data)

(0.941760012507842, 0.8508816764850232)

In [9]:
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# Parameter Tuning
model = xgb.XGBRegressor()
param_dist = {"max_depth": [10,30,50],
              "min_child_weight" : [1,3,6],
              "n_estimators": [100],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(train, y_train)

model = grid_search.best_estimator_
print(model)

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200,\
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(train,y_train)

auc(model, train, test)

NameError: name 'train' is not defined

In [ ]:
d_train = lgb.Dataset(train, label=y_train)
params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}

# Without Categorical Features
model2 = lgb.train(params, d_train)
auc2(model2, train, test)

#With Catgeorical Features
cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
auc2(model2, train, test)

In [ ]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = cb.CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(train, y_train)

With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train)
auc(clf, train, test)

With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train, cat_features= cat_features_index)
auc(clf, train, test)